In [1]:
import numpy as np 
import pandas as pd 

In [2]:
exclude_genes = pd.read_csv("./list_of_excluded_genes.csv")
raw_df = pd.read_csv("../../Final_expression_051820/Sample_final_df_RawExp.csv")
full_mapping = pd.read_table('./ATCC_CFT073_K12MG_K12DH_BW_gene_map.txt')
full_mapping.columns = ['ATCC', 'ATCC_names', 'BW', 'BW_names', 'DH', 'DH_names', 'MG', 'MG_names', 'drop']
full_mapping = full_mapping[['ATCC_names','BW_names', 'DH_names', 'MG_names']]
full_mapping = full_mapping.drop_duplicates(keep='first') 

In [3]:
#140 genes to exclude
ATCC_exclude = exclude_genes['ATCC_names'].drop_duplicates().to_numpy() #138
DH_exclude = exclude_genes['DH_names'].drop_duplicates().to_numpy() #136
BW_exclude = exclude_genes['BW_names'].drop_duplicates().to_numpy() #136
MG_exclude = exclude_genes['MG_names'].drop_duplicates().to_numpy() #136

In [4]:
# filter individual strains
ATCC_raw = raw_df[raw_df['strain']=='ATCC'] #3181
BW_raw = raw_df[raw_df['strain']=='BW25113'] #3040
DH_raw = raw_df[raw_df['strain']=='K12DH'] #3058
MG_raw = raw_df[raw_df['strain']=='K12MG'] #2969

In [55]:
print(len(ATCC_norm_new), len(BW_norm_new), len(DH_norm_new), len(MG_norm_new))

3066 2925 2944 2869


#### fixing DH

In [6]:
DH_info = pd.read_table("../Ecoli_K12_DH10B_NC_010473_gene_info.txt")

In [7]:
DH_raw = pd.merge(DH_raw, DH_info[['gbk_locus_tag','gbk_old_locus_tag']],
                 left_on='Locus_tag', right_on='gbk_old_locus_tag', how='left')

In [8]:
DH_raw.drop(['Locus_tag','gbk_old_locus_tag'], axis=1, inplace=True)
DH_raw.rename(columns={'gbk_locus_tag':'Locus_tag'}, inplace=True)

In [9]:
ATCC_raw.rename(columns={'Locus_tag':'gene_id', 'gene_id':'Locus_tag'}, inplace=True)

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:4295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


#### exclude list of genes for every strain

In [10]:
ATCC_new = ATCC_raw[~ATCC_raw['Locus_tag'].isin(ATCC_exclude)].reset_index(drop=True) #3181
BW_new = BW_raw[~BW_raw['Locus_tag'].isin(MG_exclude)].reset_index(drop=True) #3040
DH_new = DH_raw[~DH_raw['Locus_tag'].isin(DH_exclude)].reset_index(drop=True) #3058
MG_new = MG_raw[~MG_raw['Locus_tag'].isin(MG_exclude)].reset_index(drop=True) #2969

In [56]:
sample_df = pd.concat([ATCC_new,BW_new,DH_new,MG_new],axis=0)
sample_df.to_csv('./Sample_final_df_RawExp.csv', index=False)

In [12]:
norm_df = pd.read_csv("../../Final_expression_051820/Sample_final_df_1025.csv")

In [13]:
# filter individual strains
ATCC_norm = norm_df[norm_df['strain']=='ATCC'] #3180
BW_norm = norm_df[norm_df['strain']=='BW25113'] #3040
DH_norm = norm_df[norm_df['strain']=='K12DH'] #3058
MG_norm = norm_df[norm_df['strain']=='K12MG'] #2969

In [14]:
ATCC_norm_new = ATCC_norm[~ATCC_norm['gene_id'].isin(ATCC_exclude)].reset_index(drop=True) #3066 
BW_norm_new = BW_norm[~BW_norm['locus_tag'].isin(MG_exclude)].reset_index(drop=True) #2925 
DH_norm_new = DH_norm[~DH_norm['gbk_locus_tag'].isin(DH_exclude)].reset_index(drop=True) #2944 
MG_norm_new = MG_norm[~MG_norm['locus_tag'].isin(MG_exclude)].reset_index(drop=True) #2869

In [57]:
norm_new_df = pd.concat([ATCC_norm_new,BW_norm_new,DH_norm_new,MG_norm_new],axis=0)
norm_new_df.to_csv('./Sample_final_df.csv', index=False)

In [16]:
dfs = [ATCC_new, BW_new, DH_new, MG_new]

for i in dfs: 
    i['exp_name'] = i['strain'] + '_' + i['replicates']
    i.drop(['strain','gbk_start', 'gbk_end', 'gbk_midpoint','gbk_strand',
            'block', 'start', 'end', 'taxa', 'replicates', 'exp', 'gene_id'], axis=1, inplace=True)

In [17]:
mapping = full_mapping[~full_mapping['ATCC_names'].isin(ATCC_exclude)]
mapping = mapping[~mapping['BW_names'].isin(BW_exclude)]

### spreading dataframes

In [18]:
ATCC_exp = ATCC_new.pivot(index='Locus_tag',columns='exp_name',values='raw_exp').reset_index()
ATCC_rev_comp = ATCC_new.pivot(index='Locus_tag',columns='exp_name',values='rev_comp').reset_index()
ATCC_inversion = ATCC_new.pivot(index='Locus_tag',columns='exp_name',values='inversion').reset_index()

In [19]:
BW_exp = BW_new.pivot(index='Locus_tag',columns='exp_name',values='raw_exp').reset_index()
BW_rev_comp = BW_new.pivot(index='Locus_tag',columns='exp_name',values='rev_comp').reset_index()
BW_inversion = BW_new.pivot(index='Locus_tag',columns='exp_name',values='inversion').reset_index()

In [20]:
DH_exp = DH_new.pivot(index='Locus_tag',columns='exp_name',values='raw_exp').reset_index()
DH_rev_comp = DH_new.pivot(index='Locus_tag',columns='exp_name',values='rev_comp').reset_index()
DH_inversion = DH_new.pivot(index='Locus_tag',columns='exp_name',values='inversion').reset_index()

In [21]:
MG_exp = MG_new.pivot(index='Locus_tag',columns='exp_name',values='raw_exp').reset_index()
MG_rev_comp = MG_new.pivot(index='Locus_tag',columns='exp_name',values='rev_comp').reset_index()
MG_inversion = MG_new.pivot(index='Locus_tag',columns='exp_name',values='inversion').reset_index()

### expression values

In [22]:
exp = pd.merge(mapping, ATCC_exp, left_on='ATCC_names', right_on='Locus_tag') 
exp.drop(['Locus_tag'], axis=1, inplace=True) 

In [23]:
exp = pd.merge(exp, BW_exp, left_on='MG_names', right_on='Locus_tag')
exp.drop(['Locus_tag'], axis=1, inplace=True) 

In [24]:
exp = pd.merge(exp, DH_exp, left_on='DH_names', right_on='Locus_tag')
exp.drop(['Locus_tag'], axis=1, inplace=True) 

In [25]:
exp = pd.merge(exp, MG_exp, left_on='MG_names', right_on='Locus_tag')
exp.drop(['Locus_tag'], axis=1, inplace=True) 

In [26]:
len(exp)

2723

### rev_comp values

In [27]:
rev_comp = pd.merge(mapping, ATCC_rev_comp, left_on='ATCC_names', right_on='Locus_tag') 
rev_comp.drop(['Locus_tag'], axis=1, inplace=True) 

In [28]:
rev_comp = pd.merge(rev_comp, BW_rev_comp, left_on='MG_names', right_on='Locus_tag')
rev_comp.drop(['Locus_tag'], axis=1, inplace=True) 

In [29]:
rev_comp = pd.merge(rev_comp, DH_rev_comp, left_on='DH_names', right_on='Locus_tag')
rev_comp.drop(['Locus_tag'], axis=1, inplace=True) 

In [30]:
rev_comp = pd.merge(rev_comp, MG_rev_comp, left_on='MG_names', right_on='Locus_tag')
rev_comp.drop(['Locus_tag'], axis=1, inplace=True) 

In [31]:
len(rev_comp)

2723

### inversion values

In [32]:
inversion = pd.merge(mapping, ATCC_inversion, left_on='ATCC_names', right_on='Locus_tag') 
inversion.drop(['Locus_tag'], axis=1, inplace=True) 

In [33]:
inversion = pd.merge(inversion, BW_inversion, left_on='MG_names', right_on='Locus_tag')
inversion.drop(['Locus_tag'], axis=1, inplace=True) 

In [34]:
inversion = pd.merge(inversion, DH_inversion, left_on='DH_names', right_on='Locus_tag')
inversion.drop(['Locus_tag'], axis=1, inplace=True) 

In [35]:
inversion = pd.merge(inversion, MG_inversion, left_on='MG_names', right_on='Locus_tag')
inversion.drop(['Locus_tag'], axis=1, inplace=True) 

In [36]:
len(inversion)

2723

### export new dfs

In [37]:
exp.to_csv('./raw_exp.csv', index=False)

In [38]:
rev_comp.to_csv('./raw_rev_comp.csv', index=False)

In [39]:
inversion.to_csv('./raw_inversion.csv', index=False)

In [40]:
len(exp) == len(rev_comp) == len(inversion)

True